# 4 Genetic Algorithm

#### Input：

    station_tasks.csv

#### Output:

    ga_stats_k*.csv
    ga_evolution_k*.png

Genetic algorithm–based vehicle routing for bike rebalancing.

Firstly, builds a distance matrix, selects a heuristic depot, and then runs a DEAP GA with random-key encoding to search for low-cost routes.

The fitness function enforces capacity, vehicle count, and full-demand-service constraints via penalties.

After optimization, the best solution is decoded into vehicle routes and printed.

In [6]:
import pandas as pd
import numpy as np
from deap import base, creator, tools, algorithms
import random
from haversine import haversine, Unit
import sys
import time
import matplotlib.pyplot as plt
from collections import defaultdict



# Load station data from station_task.csv
try:
    df_task = pd.read_csv("station_tasks.csv")
except Exception as e:
    sys.exit(1)

df_task["station_id"] = df_task["station_id"].astype(str)
df_task["move_out"] = df_task["move_out"].astype(int)
df_task["move_in"] = df_task["move_in"].astype(int)
df_task["initial_stock"] = df_task["initial_stock"].astype(int)
df_task["capacity"] = df_task["capacity"].astype(int)
df_task["net"] = df_task["net"].astype(int)


# Data cleaning
missing_coords = df_task[df_task[['latitude', 'longitude']].isna().any(axis=1)]

df_station_location = df_task.dropna(subset=['latitude', 'longitude']).copy()
df_station_clean = df_station_location[df_station_location['net'] != 0].copy()

STATIONS = df_station_clean['station_id'].astype(str).tolist()
print(f"Loaded number of stations: {len(STATIONS)}")

# Index mapping
id_to_int = {sid: idx for idx, sid in enumerate(STATIONS)}
int_to_id = {idx: sid for sid, idx in id_to_int.items()}
STATIONS_INT = list(id_to_int.values())
N_STATIONS = len(STATIONS_INT)

# Data dictionaries
initial_stock = {id_to_int[row.station_id]: row.initial_stock for row in df_station_clean.itertuples()}
capacity = {id_to_int[row.station_id]: row.capacity for row in df_station_clean.itertuples()}
demand_U = {id_to_int[row.station_id]: row.move_out for row in df_station_clean.itertuples()}
demand_D = {id_to_int[row.station_id]: row.move_in for row in df_station_clean.itertuples()}
Net_Demand = {id_to_int[row.station_id]: row.net for row in df_station_clean.itertuples()}

station_coords_raw = {
    row.station_id: {"lat": float(row.latitude), "lon": float(row.longitude)}
    for row in df_station_clean.itertuples()
}

# Distance matrix
distance_matrix = pd.DataFrame(index=STATIONS_INT, columns=STATIONS_INT, dtype=float)
for i in STATIONS_INT:
    for j in STATIONS_INT:
        if i == j:
            distance_matrix.loc[i, j] = 0.0
        else:
            sid_i, sid_j = int_to_id[i], int_to_id[j]
            coord_i = (station_coords_raw[sid_i]['lat'], station_coords_raw[sid_i]['lon'])
            coord_j = (station_coords_raw[sid_j]['lat'], station_coords_raw[sid_j]['lon'])
            distance_matrix.loc[i, j] = haversine(coord_i, coord_j, unit=Unit.KILOMETERS)

# Save matrix
distance_matrix.to_csv('distance_matrix_int_km.csv')
print("Distance matrix saved.")

# Best Depot
best_depot_int = min(STATIONS_INT, key=lambda x: distance_matrix.loc[x, :].sum())
OPTIMAL_CANDIDATE_DEPOTS_INT = [best_depot_int]
print(f"optimal Depot: {int_to_id[best_depot_int]}")

# Parameters
Cost_Movement_KM = 0.8
Cost_Fixed_Vehicle = 0
Vehicle_Capacity_Dict = {1: 30, 2: 30, 3: 30, 4: 30, 5: 30, 6: 30, 7: 40, 8: 40, 9: 40, 10: 40}

print(f"Data preparation finished. Total valid stations: {N_STATIONS}.")


# --- DEAP definition ---
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()


# Individual creation function
def create_permutation_individual(ind_class, dimension):
    """Create a permutation individual [0, dimension), no duplicates."""
    genes = list(range(dimension))
    random.shuffle(genes)
    return ind_class(genes)


# Order Crossover
def order_crossover(ind1, ind2):
    """Order crossover: preserve relative order and avoid duplicates."""
    size = len(ind1)
    cx_point1 = random.randint(0, size - 2)
    cx_point2 = random.randint(cx_point1 + 1, size)
    
    child1, child2 = [-1] * size, [-1] * size
    child1[cx_point1:cx_point2] = ind1[cx_point1:cx_point2]
    child2[cx_point1:cx_point2] = ind2[cx_point1:cx_point2]
    
    segment1 = set(ind1[cx_point1:cx_point2])
    remaining1 = [gene for gene in ind2 if gene not in segment1]
    idx = 0
    for i in range(size):
        if child1[i] == -1:
            child1[i] = remaining1[idx]
            idx += 1
    
    segment2 = set(ind2[cx_point1:cx_point2])
    remaining2 = [gene for gene in ind1 if gene not in segment2]
    idx = 0
    for i in range(size):
        if child2[i] == -1:
            child2[i] = remaining2[idx]
            idx += 1
    
    ind1[:], ind2[:] = child1, child2
    return ind1, ind2


# Mutation operators
def swap_mutation(individual, indpb):
    """Swap mutation: randomly swap gene pairs."""
    size = len(individual)
    for i in range(size):
        if random.random() < indpb:
            j = random.randint(0, size - 1)
            individual[i], individual[j] = individual[j], individual[i]
    return (individual,)


def inversion_mutation(individual, indpb):
    """Inversion mutation: reverse a random segment."""
    if random.random() < indpb:
        size = len(individual)
        point1 = random.randint(0, size - 2)
        point2 = random.randint(point1 + 1, size)
        individual[point1:point2] = individual[point1:point2][::-1]
    return (individual,)


# Chromosome decoding
def decode_chromosome_to_paths(chromosome, genes_to_permute, depot_int):
    """Decode a permutation chromosome into vehicle paths."""
    ordered_path_list = [genes_to_permute[i] for i in chromosome]
    
    paths_by_vehicle_int = {}
    current_path = [depot_int]
    current_k = 1
    
    for station_id_int in ordered_path_list:
        if station_id_int == depot_int:
            current_path.append(depot_int)
            paths_by_vehicle_int[current_k] = current_path
            current_k += 1
            current_path = [depot_int]
        else:
            current_path.append(station_id_int)
    
    current_path.append(depot_int)
    paths_by_vehicle_int[current_k] = current_path
    
    return paths_by_vehicle_int


#  fitness function

# Soft-constraint penalty coefficients
PENALTY_LOAD_NEGATIVE = 500.0      
PENALTY_LOAD_OVER_CAPACITY = 500.0  
PENALTY_EXCESS_VEHICLE = 5000.0     
PENALTY_UNSERVED_STATION = 10000.0  

# Debug counters
debug_counters = {
    'total_evaluations': 0,
    'feasible_solutions': 0,
    'load_negative_count': 0,
    'load_over_capacity_count': 0,
    'excess_vehicle_count': 0,
    'unserved_station_count': 0,
}


def calculate_vrp_fitness_soft(paths_by_vehicle_int, K, distance_matrix,
                                Vehicle_Capacity_Dict, demand_U, demand_D,
                                Net_Demand, depot_int, Cost_Movement_KM, MAX_K_ALLOWED):
    """
    Soft-constraint VRP fitness function:
    - GA can gradually improve from infeasible solutions.
    Returns:
        fitness_value, is_feasible, penalty_details
    """
    global debug_counters
    debug_counters['total_evaluations'] += 1
    
    required_stations = set(demand_U.keys())
    if depot_int in required_stations:
        required_stations.remove(depot_int)
    
    cost_travel = 0.0
    penalty_load = 0.0
    penalty_vehicle = 0.0
    penalty_unserved = 0.0
    
    served_stations = set()
    used_vehicles_count = 0
    
    penalty_details = {
        'load_negative': 0.0,
        'load_over_capacity': 0.0,
        'excess_vehicles': 0.0,
        'unserved_stations': 0.0,
    }
    
    # Loop the path of each vehicle
    for k_id, path in paths_by_vehicle_int.items():
        if len(path) > 2:
            used_vehicles_count += 1
            
            # read vehicle capacity
            vehicle_capacity = Vehicle_Capacity_Dict.get(k_id, 30)
            
            current_load = 0
            
            # # Iterate through the path
            for i in range(len(path) - 1):
                start_node = path[i]
                end_node = path[i + 1]
                
                # Add travel cost
                distance = distance_matrix.loc[start_node, end_node]
                cost_travel += distance * Cost_Movement_KM
                
                # constraint：every station's demand must be satisfied
                if end_node != depot_int:
                    served_stations.add(end_node)
                    
                    # updata load
                    service_amount = Net_Demand.get(end_node, 0)
                    current_load += service_amount
                    
                    # # Soft constraint: penalty for negative load
                    if current_load < 0:
                        violation = abs(current_load)
                        penalty_load += violation * PENALTY_LOAD_NEGATIVE
                        penalty_details['load_negative'] += violation
                        debug_counters['load_negative_count'] += 1
                    
                    # # Soft constraint: overload penalty
                    if current_load > vehicle_capacity:
                        violation = current_load - vehicle_capacity
                        penalty_load += violation * PENALTY_LOAD_OVER_CAPACITY
                        penalty_details['load_over_capacity'] += violation
                        debug_counters['load_over_capacity_count'] += 1
    

    
    # Soft constraint: penalty for unserved stations
    unserved = required_stations - served_stations
    if unserved:
        penalty_unserved = len(unserved) * PENALTY_UNSERVED_STATION
        penalty_details['unserved_stations'] = len(unserved)
        debug_counters['unserved_station_count'] += 1
    
    # Total cost for reblancing
    total_penalty = penalty_load + penalty_vehicle + penalty_unserved
    total_cost = cost_travel + total_penalty
    
    # Check feasibility
    is_feasible = (total_penalty == 0)
    if is_feasible:
        debug_counters['feasible_solutions'] += 1
    
    # # Compute fitness 
    if total_cost > 0:
        fitness = 1.0 / total_cost
    else:
        fitness = 1e9
    
    return fitness, is_feasible, penalty_details


# Statistics recorder class
class GenerationStats:
    """Record statistical data for each generation"""
    
    def __init__(self):
        self.generations = []
        self.min_fitness = []
        self.max_fitness = []
        self.avg_fitness = []
        self.std_fitness = []
        self.min_cost = []
        self.max_cost = []
        self.avg_cost = []
        self.feasible_count = []
        self.feasible_ratio = []
        self.best_feasible_cost = []
    
    def record(self, gen, population, pop_size):
        fits = [ind.fitness.values[0] for ind in population]
        costs = [1.0 / f if f > 0 else float('inf') for f in fits]
        
        # Filter out infinite cost values
        valid_costs = [c for c in costs if c < 1e8]
        
        self.generations.append(gen)
        self.min_fitness.append(min(fits))
        self.max_fitness.append(max(fits))
        self.avg_fitness.append(np.mean(fits))
        self.std_fitness.append(np.std(fits))
        
        if valid_costs:
            self.min_cost.append(min(valid_costs))
            self.max_cost.append(max(valid_costs))
            self.avg_cost.append(np.mean(valid_costs))
        else:
            self.min_cost.append(float('inf'))
            self.max_cost.append(float('inf'))
            self.avg_cost.append(float('inf'))
        
        # feasible solution count
        feasible = [c for c in costs if c < 1e6]
        self.feasible_count.append(len(feasible))
        self.feasible_ratio.append(len(feasible) / pop_size * 100)
        
        if feasible:
            self.best_feasible_cost.append(min(feasible))
        else:
            self.best_feasible_cost.append(float('inf'))
    
    def to_dataframe(self):
        return pd.DataFrame({
            'generation': self.generations,
            'min_fitness': self.min_fitness,
            'max_fitness': self.max_fitness,
            'avg_fitness': self.avg_fitness,
            'std_fitness': self.std_fitness,
            'min_cost': self.min_cost,
            'max_cost': self.max_cost,
            'avg_cost': self.avg_cost,
            'feasible_count': self.feasible_count,
            'feasible_ratio': self.feasible_ratio,
            'best_feasible_cost': self.best_feasible_cost,
        })


# Visualization functions
def plot_ga_evolution(stats, k_vehicles, depot_id, save_path=None):
    """    
       Plot GA evolution curves.

    """
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle(f'GA Evolution: Depot={depot_id}, K={k_vehicles} Vehicles', fontsize=14, fontweight='bold')
    
    generations = stats.generations
    
    # Plot 1: Fitness evolution
    ax1 = axes[0, 0]
    ax1.plot(generations, stats.max_fitness, 'g-', linewidth=2, label='Max Fitness (Best)')
    ax1.plot(generations, stats.avg_fitness, 'b-', linewidth=1.5, label='Avg Fitness')
    ax1.plot(generations, stats.min_fitness, 'r--', linewidth=1, alpha=0.7, label='Min Fitness (Worst)')
    ax1.fill_between(generations, 
                     np.array(stats.avg_fitness) - np.array(stats.std_fitness),
                     np.array(stats.avg_fitness) + np.array(stats.std_fitness),
                     alpha=0.2, color='blue', label='±1 Std Dev')
    ax1.set_xlabel('Generation')
    ax1.set_ylabel('Fitness (1/Cost)')
    ax1.set_title('Fitness Evolution')
    ax1.legend(loc='lower right')
    ax1.grid(True, alpha=0.3)
    ax1.set_yscale('log') 
    

    # Cost evolution
    ax2 = axes[0, 1]
    
    valid_min_cost = [c if c < 1e8 else np.nan for c in stats.min_cost]
    valid_avg_cost = [c if c < 1e8 else np.nan for c in stats.avg_cost]
    valid_best_feasible = [c if c < 1e8 else np.nan for c in stats.best_feasible_cost]
    
    ax2.plot(generations, valid_min_cost, 'g-', linewidth=2, label='Min Cost (Best)')
    ax2.plot(generations, valid_avg_cost, 'b-', linewidth=1.5, label='Avg Cost')
    ax2.plot(generations, valid_best_feasible, 'r-', linewidth=2, linestyle='--', label='Best Feasible Cost')
    ax2.set_xlabel('Generation')
    ax2.set_ylabel('Cost')
    ax2.set_title('Cost Evolution')
    ax2.legend(loc='upper right')
    ax2.grid(True, alpha=0.3)
    
    # Feasible ratio 
    ax3 = axes[1, 0]
    ax3.plot(generations, stats.feasible_ratio, 'purple', linewidth=2)
    ax3.fill_between(generations, 0, stats.feasible_ratio, alpha=0.3, color='purple')
    ax3.set_xlabel('Generation')
    ax3.set_ylabel('Feasible Solutions (%)')
    ax3.set_title('Feasible Solution Ratio')
    ax3.set_ylim(0, 105)
    ax3.axhline(y=50, color='orange', linestyle='--', alpha=0.5, label='50% threshold')
    ax3.axhline(y=90, color='green', linestyle='--', alpha=0.5, label='90% threshold')
    ax3.legend(loc='lower right')
    ax3.grid(True, alpha=0.3)
    
    # Feasible solution count
    ax4 = axes[1, 1]
    ax4.bar(generations, stats.feasible_count, color='teal', alpha=0.7, width=1.0)
    ax4.set_xlabel('Generation')
    ax4.set_ylabel('Number of Feasible Solutions')
    ax4.set_title('Feasible Solution Count per Generation')
    ax4.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f"figures saved: {save_path}")
    
    plt.close()
    
    return fig





# GA main loop
ga_parameters = {
    'max_num_iteration': 200,
    'population_size': 300,
    'mutation_probability': 0.15,
    'crossover_probability': 0.85,
}

max_vehicles = 3
best_overall_cost = float('inf')
best_solution = {}
all_k_stats = {}  

start_time = time.time()

for depot_int in OPTIMAL_CANDIDATE_DEPOTS_INT:
    stations_to_visit_int = [i for i in STATIONS_INT if i != depot_int]
    depot_id = int_to_id[depot_int]
    
    for k in range(1, max_vehicles + 1):
        print(f"--- GA Running: Depot={depot_id}, K={k} Vehicles ---")
        
        # Reset counters
        for key in debug_counters:
            debug_counters[key] = 0
        
        genes_to_permute = stations_to_visit_int + [depot_int] * (k - 1)
        dimension = len(genes_to_permute)
        
        if dimension == 0:
            continue
        
        # Fitness
        def fitness_wrapper_deap(chromosome,
                                  genes_pool=genes_to_permute,
                                  depot=depot_int,
                                  k_vehicles=k):
            fitness, _, _ = calculate_vrp_fitness_soft(
                paths_by_vehicle_int=decode_chromosome_to_paths(chromosome, genes_pool, depot),
                K=k_vehicles,
                distance_matrix=distance_matrix,
                Vehicle_Capacity_Dict=Vehicle_Capacity_Dict,
                demand_U=demand_U,
                demand_D=demand_D,
                Net_Demand=Net_Demand,
                depot_int=depot,
                Cost_Movement_KM=Cost_Movement_KM,
                MAX_K_ALLOWED=k_vehicles
            )
            return (fitness,)
        
        # register toolbox
        for op in ["individual", "population", "evaluate", "mate", "mutate", "select"]:
            if op in toolbox.__dict__:
                toolbox.unregister(op)
        
        toolbox.register("individual", create_permutation_individual, creator.Individual, dimension=dimension)
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)
        toolbox.register("evaluate", fitness_wrapper_deap)
        toolbox.register("mate", order_crossover)
        toolbox.register("mutate", swap_mutation, indpb=0.05)
        toolbox.register("select", tools.selTournament, tournsize=3)
        
        # Initialize
        pop = toolbox.population(n=ga_parameters['population_size'])
        hof = tools.HallOfFame(5)  # Save top 5 solution
        gen_stats = GenerationStats()
        
        # Evaluating initial population
        print("Evaluating initial population...")
        fitnesses = list(map(toolbox.evaluate, pop))
        for ind, fit in zip(pop, fitnesses):
            ind.fitness.values = fit
        
        hof.update(pop)
        gen_stats.record(0, pop, ga_parameters['population_size'])
        
        print(f"Initial population: Best Cost={gen_stats.min_cost[0]:.2f}, "
              f"Feasible={gen_stats.feasible_count[0]} ({gen_stats.feasible_ratio[0]:.1f}%)")
        

        #  evolution loop
        print("Starting evolution...")
        
        for gen in range(1, ga_parameters['max_num_iteration'] + 1):

            offspring = toolbox.select(pop, len(pop))
            offspring = [creator.Individual(ind[:]) for ind in offspring]
            
            for i in range(0, len(offspring) - 1, 2):
                if random.random() < ga_parameters['crossover_probability']:
                    toolbox.mate(offspring[i], offspring[i + 1])
                    del offspring[i].fitness.values
                    del offspring[i + 1].fitness.values
            
            for ind in offspring:
                if random.random() < ga_parameters['mutation_probability']:
                    toolbox.mutate(ind)
                    del ind.fitness.values
            
            invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
            fitnesses = list(map(toolbox.evaluate, invalid_ind))
            for ind, fit in zip(invalid_ind, fitnesses):
                ind.fitness.values = fit
            
            pop[:] = offspring
            hof.update(pop)
            gen_stats.record(gen, pop, ga_parameters['population_size'])
            
            
            if gen % 20 == 0 or gen == ga_parameters['max_num_iteration']:
                print(f"Gen {gen:4d}: MIn cost={gen_stats.min_cost[-1]:.2f}, "
                      f"Feasible={gen_stats.feasible_ratio[-1]:.1f}%")
        

        all_k_stats[k] = gen_stats
        
        
        
        # Polt GA evolution
        plot_filename = f'ga_evolution_k{k}.png'
        plot_ga_evolution(gen_stats, k, depot_id, save_path=plot_filename)
        
        # Get optimal solution
        if hof:
            best_chromosome = hof[0]
            best_fitness = hof[0].fitness.values[0]
            current_cost = 1 / best_fitness if best_fitness > 0 else float('inf')
            
            # Check optmial solution
            best_paths = decode_chromosome_to_paths(best_chromosome, genes_to_permute, depot_int)
            _, is_feasible, penalty_details = calculate_vrp_fitness_soft(
                best_paths, k, distance_matrix, Vehicle_Capacity_Dict,
                demand_U, demand_D, Net_Demand, depot_int, Cost_Movement_KM, k
            )
            
            print(f"\n --- K={k} Optimal Solution ---")
            print(f"cost: {current_cost:.2f}")
            print(f"Feasibility: {'feasible' if is_feasible else 'not feasible'}")
            if not is_feasible:
                print(f"penalty details: {penalty_details}")
        else:
            current_cost = float('inf')
        
        # Record the global best solution
        if current_cost < best_overall_cost:
            best_overall_cost = current_cost
            best_solution = {
                'Depot_Int': depot_int,
                'K_Vehicles': k,
                'Total_Cost': current_cost,
                'Optimal_Chromosome': list(best_chromosome) if hof else None,
                'Genes_Pool': genes_to_permute.copy(),
                'Is_Feasible': is_feasible,
            }
        



# Final output
end_time = time.time()
print(f"/n--- Optimization Completed (Time: {end_time - start_time:.2f} s) ---")

if best_solution:
    print(f"\n Best Global Solution:")
    print(f"  Depot: {int_to_id[best_solution['Depot_Int']]}")
    print(f"  Vehicles: {best_solution['K_Vehicles']}")
    print(f"  Total cost: {best_solution['Total_Cost']:.2f}")
    print(f"  Feasible: {'true' if best_solution['Is_Feasible'] else 'false'}")
    
    # path
    if best_solution['Optimal_Chromosome']:
        final_paths = decode_chromosome_to_paths(
            best_solution['Optimal_Chromosome'],
            best_solution['Genes_Pool'],
            best_solution['Depot_Int']
        )
        
        print("\n--- Optimal Path ---")
        total_distance = 0.0
        for k_id, path_int in final_paths.items():
            if len(path_int) > 2:
                path_str = [int_to_id[i] for i in path_int]
                path_distance = sum(
                    distance_matrix.loc[path_int[i], path_int[i + 1]]
                    for i in range(len(path_int) - 1)
                )
                print(f"vechile {k_id}: {' -> '.join(path_str)}")
                print(f" distance: {path_distance:.2f} km")
                total_distance += path_distance
            else:
                print(f"vechile {k_id}: didn't use")
        
        print(f"\n total distance: {total_distance:.2f} km")
        print(f"total cost: {total_distance * Cost_Movement_KM:.2f}")



Loaded number of stations: 106
Distance matrix saved.
optimal Depot: 1019
Data preparation finished. Total valid stations: 106.
--- GA Running: Depot=1019, K=1 Vehicles ---
Evaluating initial population...


/opt/anaconda3/envs/Xpress/lib/python3.12/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/opt/anaconda3/envs/Xpress/lib/python3.12/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Initial population: Best Cost=242.77, Feasible=294 (98.0%)
Starting evolution...
Gen   20: MIn cost=218.08, Feasible=100.0%
Gen   40: MIn cost=212.77, Feasible=100.0%
Gen   60: MIn cost=201.46, Feasible=100.0%
Gen   80: MIn cost=178.41, Feasible=100.0%
Gen  100: MIn cost=158.65, Feasible=100.0%
Gen  120: MIn cost=151.31, Feasible=100.0%
Gen  140: MIn cost=149.99, Feasible=100.0%
Gen  160: MIn cost=145.90, Feasible=100.0%
Gen  180: MIn cost=139.56, Feasible=100.0%
Gen  200: MIn cost=138.92, Feasible=100.0%


/var/folders/37/c8t1zshs29x5t669m6kp50nm0000gn/T/ipykernel_79781/499905511.py:393: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "r-" (-> linestyle='-'). The keyword argument will take precedence.
  ax2.plot(generations, valid_best_feasible, 'r-', linewidth=2, linestyle='--', label='Best Feasible Cost')


figures saved: ga_evolution_k1.png

 --- K=1 Optimal Solution ---
cost: 138.50
Feasibility: feasible
--- GA Running: Depot=1019, K=2 Vehicles ---
Evaluating initial population...
Initial population: Best Cost=4743.79, Feasible=297 (99.0%)
Starting evolution...
Gen   20: MIn cost=227.69, Feasible=100.0%
Gen   40: MIn cost=194.78, Feasible=100.0%
Gen   60: MIn cost=177.97, Feasible=100.0%
Gen   80: MIn cost=158.94, Feasible=100.0%
Gen  100: MIn cost=147.99, Feasible=100.0%
Gen  120: MIn cost=133.73, Feasible=100.0%
Gen  140: MIn cost=127.74, Feasible=100.0%
Gen  160: MIn cost=126.45, Feasible=100.0%
Gen  180: MIn cost=118.36, Feasible=100.0%
Gen  200: MIn cost=117.81, Feasible=100.0%
figures saved: ga_evolution_k2.png

 --- K=2 Optimal Solution ---
cost: 117.81
Feasibility: feasible
--- GA Running: Depot=1019, K=3 Vehicles ---
Evaluating initial population...
Initial population: Best Cost=12256.40, Feasible=300 (100.0%)
Starting evolution...
Gen   20: MIn cost=228.97, Feasible=100.0%
Gen


Best solution saved:

figures saved: ga_evolution_k1.png

 --- K=1 Optimal Solution ---
cost: 138.50
Feasibility: feasible
--- GA Running: Depot=1019, K=2 Vehicles ---
Evaluating initial population...
Initial population: Best Cost=4743.79, Feasible=297 (99.0%)
Starting evolution...
Gen   20: MIn cost=227.69, Feasible=100.0%
Gen   40: MIn cost=194.78, Feasible=100.0%
Gen   60: MIn cost=177.97, Feasible=100.0%
Gen   80: MIn cost=158.94, Feasible=100.0%
Gen  100: MIn cost=147.99, Feasible=100.0%
Gen  120: MIn cost=133.73, Feasible=100.0%
Gen  140: MIn cost=127.74, Feasible=100.0%
Gen  160: MIn cost=126.45, Feasible=100.0%
Gen  180: MIn cost=118.36, Feasible=100.0%
Gen  200: MIn cost=117.81, Feasible=100.0%
figures saved: ga_evolution_k2.png

 --- K=2 Optimal Solution ---
cost: 117.81
Feasibility: feasible
--- GA Running: Depot=1019, K=3 Vehicles ---
Evaluating initial population...
Initial population: Best Cost=12256.40, Feasible=300 (100.0%)
Starting evolution...
Gen   20: MIn cost=228.97, Feasible=100.0%
Gen   40: MIn cost=203.86, Feasible=100.0%
Gen   60: MIn cost=191.72, Feasible=100.0%
Gen   80: MIn cost=161.41, Feasible=100.0%
Gen  100: MIn cost=145.53, Feasible=100.0%
Gen  120: MIn cost=127.51, Feasible=100.0%
Gen  140: MIn cost=116.04, Feasible=100.0%
Gen  160: MIn cost=107.97, Feasible=100.0%
Gen  180: MIn cost=106.98, Feasible=100.0%
Gen  200: MIn cost=106.98, Feasible=100.0%
figures saved: ga_evolution_k3.png

 --- K=3 Optimal Solution ---
cost: 106.98
Feasibility: feasible
/n--- Optimization Completed (Time: 46.63 s) ---

 Best Global Solution:
  Depot: 1019
  Vehicles: 3
  Total cost: 106.98
  Feasible: true

--- Optimal Path ---
vechile 1: didn't use
vechile 2: 1019 -> 252 -> 255 -> 265 -> 260 -> 289 -> 358 -> 259 -> 262 -> 1038 -> 1728 -> 1813 -> 1800 -> 253 -> 359 -> 344 -> 1748 -> 290 -> 352 -> 877 -> 1766 -> 1025 -> 351 -> 1823 -> 1767 -> 1744 -> 346 -> 1097 -> 246 -> 1798 -> 256 -> 870 -> 1818 -> 1017 -> 248 -> 1765 -> 1050 -> 1098 -> 1727 -> 225 -> 1824 -> 284 -> 1026 -> 2259 -> 247 -> 1821 -> 1093 -> 349 -> 1859 -> 1822 -> 1860 -> 1747 -> 876 -> 1799 -> 1055 -> 1808 -> 1738 -> 885 -> 1102 -> 296 -> 1756 -> 354 -> 285 -> 1743 -> 1737 -> 1745 -> 1730 -> 356 -> 273 -> 1769 -> 879 -> 1024 -> 1051 -> 1092 -> 1039 -> 881 -> 1028 -> 189 -> 251 -> 1731 -> 1726 -> 1768 -> 345 -> 1757 -> 889 -> 1807 -> 1764 -> 1763 -> 355 -> 1720 -> 820 -> 261 -> 1090 -> 2268 -> 183 -> 890 -> 1096 -> 171 -> 340 -> 1721 -> 275 -> 249 -> 342 -> 343 -> 264 -> 1052 -> 1019
 distance: 133.72 km
vechile 3: didn't use

 total distance: 133.72 km
total cost: 106.98
